#### MNE  preproc EEG resting state 

In [1]:
#Import some basic libraries first 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path as op
import time
from time import sleep 
from time import time
from joblib import Parallel, delayed
#import MNE 
import mne
from mne.channels.montage import get_builtin_montages
from mne.time_frequency import tfr_morlet #, psd_welch

from mne import io
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs,
                               create_eog_epochs)



In [10]:
# Get directories 
data_path = '/add/your/path/here'
save_path = '/add/your/path/here'

In [20]:
# Specify if you want to plot preproc figures
plot_figures = 0 
# Specify if you want to run ICA 
run_ica = 1
# Specify the day 
day = "DAY1"
#Specify navigation
EEGtask = "navigateEEG2"

navigate_files = {"DAY1": [], "DAY2": []}
participant_ids = []

In [21]:
#First create all the files that need to be preprocessed 
for participant_folder in os.listdir(data_path):
    participant_id = participant_folder.split('-')[1]
    
    participant_ids.append(participant_id)
    
    navigate_filename = f"sub-{participant_id}_{EEGtask}.bdf"
    session_filename = f"sub-{participant_id}_ses-{day}"
    navigate_file = os.path.join(data_path, participant_folder, session_filename, navigate_filename)
    
    navigate_files[day].append(navigate_file)

print(navigate_files[day])

['/Volumes/DAV/NovMemEEG/EEG_data_test/sub-051/sub-051_ses-DAY1/sub-051_navigateEEG2.bdf', '/Volumes/DAV/NovMemEEG/EEG_data_test/sub-052/sub-052_ses-DAY1/sub-052_navigateEEG2.bdf']


In [22]:
def run_preproc(filename):
    rawrseeg = mne.io.read_raw_bdf(filename, preload=True, verbose=None) #
    
    # Downsample
    rsEEG_data = rawrseeg.copy().resample(sfreq=250)
    
    # set channel types
    rsEEG_data.set_channel_types({'EXG1': 'eog',
                                 'EXG2': 'eog',
                                 'EXG3': 'eog',
                                 'EXG4': 'eog',
                                 'EXG5': 'eog',
                                 'EXG6': 'eog',
                                 'EXG7': 'eog',
                                 'EXG8': 'eog',})
    EOG_chans = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6']
    
    # Set reference
    rsEEG_data = rsEEG_data.set_eeg_reference(ref_channels=['EXG5', 'EXG6'],
                                          projection=False, verbose=False)

    
    # Set channel montage
    chs = mne.channels.make_standard_montage('standard_1020')
    rsEEG_data.set_montage(chs, verbose=False);
    if plot_figures:
        rsEEG_data.plot_sensors(kind='3d', ch_type='eeg')
        
    # Filter 
    rsEEG_data = rsEEG_data.filter(l_freq=0.5,h_freq=48)
    
    # RUN ICA 
    if run_ica:

        #ICA settings 
        method = 'picard'
        max_iter = 500
        fit_params = dict(fastica_it=5)
        random_state = 97 # ICA fitting is not deterministic and ICs may not always be returned in the same order, so random state/seed indicates you get same results every time your run 

        ica = mne.preprocessing.ICA(n_components=30, 
                                    max_iter=max_iter, 
                                    method=method,
                                    fit_params=fit_params,
                                    random_state=random_state)
        ica.fit(rsEEG_data)
    else:
        print("ICA already run and loading ICA data")
        #ica = read_ica(os.path.join(RESULTS_PATH, 'ICA', subj_label + '-ica.fif'))
        
    # Automated ICA component rejection: drop ICs that correlate with EOG
    ica.exclude = []

    drop_inds = []
    for chi in EOG_chans:
        inds, scores = ica.find_bads_eog(rsEEG_data, ch_name=chi, threshold=2.5,
                                         l_freq=1, h_freq=10, verbose=False)
        drop_inds.extend(inds)
        if plot_figures:
            ica.plot_scores(scores, exclude=inds, labels=chi);

    drop_inds = list(set(drop_inds))
    
    # Check the set of components to be dropped
    ica.exclude = drop_inds
    
    if plot_figures: 
        # Check the set of components to be dropped
        print('Number of component to drop: \t', len(ica.exclude))
        print('Dropped component indices: \t', ica.exclude)
    
    # Apply ICA to data
    rsEEG_data = ica.apply(rsEEG_data);
    
    # Check the overlay of the data, with the components to be rejected
    if plot_figures: 
        ica.plot_overlay(rsEEG_data, exclude=drop_inds);
    
    # Create events 
    events = mne.make_fixed_length_events(rsEEG_data, id=1, start=3, 
                                          stop=None, duration=3, 
                                          overlap = 0.0)
    
    if plot_figures:
        #plot the events 
        mne.viz.plot_events(events[:600])
    
    # define event in rsEEG
    event_ids = {"navigate": 1}
    
    # Create epochs
    tmin = -0.5
    tmax = 2
    event_id = event_ids

    epochs = mne.Epochs(rsEEG_data, events, 
                        event_id = event_ids, 
                        tmin=tmin, tmax=tmax, 
                        baseline=None, 
                        detrend=1)
    return epochs 


In [23]:
# 

In [24]:
#Run the preprocessing function in Parallel 
t0 = time()
batch_size = 5
epochs_preproc = []

for i in range(0, len(navigate_files[day]), batch_size):
    batch_files = navigate_files[day][i:i+batch_size]

    batch_epochs_preproc = Parallel(n_jobs=8)(delayed(run_preproc)(eeg_file) for eeg_file in batch_files)

    epochs_preproc.extend(batch_epochs_preproc)
    
    # Take a 40-second break before processing the next batch
    if i + batch_size < len(navigate_files[day]):
        print("Taking a 40-second break...")
        sleep(40)
t1 = time()
print(f'This MNE preproc code run in parallel took {t1 - t0} seconds to run')

Extracting EDF parameters from /Volumes/DAV/NovMemEEG/EEG_data_test/sub-051/sub-051_ses-DAY1/sub-051_navigateEEG2.bdf...
Extracting EDF parameters from /Volumes/DAV/NovMemEEG/EEG_data_test/sub-052/sub-052_ses-DAY1/sub-052_navigateEEG2.bdf...
BDF file detected
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 319487  =      0.000 ...   156.000 secs...
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 311295  =      0.000 ...   152.000 secs...
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
1 events found
Event IDs: [65536]
Trigger channel has a non-zero initial value of 73728 (consider using initial_event=True to detect this event)
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.0s finished


Selecting by number: 30 components
Trigger channel has a non-zero initial value of 73728 (consider using initial_event=True to detect this event)
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 48 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 48.00 Hz
- Upper transition bandwidth: 12.00 Hz (-6 dB cutoff frequency: 54.00 Hz)
- Filter length: 1651 samples (6.604 sec)

Fitting ICA to data using 32 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.0s finished


Selecting by number: 30 components
Fitting ICA took 3.1s.
Applying ICA to Raw instance
    Transforming to ICA space (30 components)
    Zeroing out 3 ICA components
    Projecting back using 32 PCA components
Not setting metadata
51 matching events found
No baseline correction applied
0 projection items activated
Fitting ICA took 4.3s.
Applying ICA to Raw instance
    Transforming to ICA space (30 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Not setting metadata
49 matching events found
No baseline correction applied
0 projection items activated
This MNE preproc code run in parallel took 8.467037200927734 seconds to run


In [25]:
# Save all the preprocessed epochs 
for j, epochs in enumerate(epochs_preproc):
    subject_id = 'sub-{}'.format(participant_ids[j])
    filename = '{}_{}_{}-epo.fif'.format(subject_id, EEGtask, day)
    filepath = os.path.join(save_path, filename)
    epochs.save(filepath, overwrite=True)
    #print(filepath)


Using data from preloaded Raw for 51 events and 626 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 626 original time points ...
Using data from preloaded Raw for 51 events and 626 original time points ...
Using data from preloaded Raw for 49 events and 626 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 626 original time points ...
Using data from preloaded Raw for 49 events and 626 original time points ...
